In [238]:
import pandas as pd
import re

## Задание 1
Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
path = 'C:\\Users\\Сергей\\'
rating= pd.read_csv(path+"ratings.csv")

In [3]:
def rating_class(param):
    if param <= 2.0:
        return 'низкий рейтинг'
    elif 4.0 >= param > 2.0:
        return 'средний рейтинг'
    elif param > 4.0:
        return 'высокий рейтинг'

In [4]:
rating['class'] = rating['rating'].apply(rating_class)

In [5]:
rating

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,средний рейтинг
100000,671,6269,4.0,1065149201,средний рейтинг
100001,671,6365,4.0,1070940363,средний рейтинг
100002,671,6385,2.5,1070979663,средний рейтинг


## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
Результат классификации запишите в отдельный столбец region.

In [124]:
keywords = pd.read_csv(path+"keywords.csv")

In [125]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [194]:
def region(column):    
    for key, values in geo_data.items():
        for value in values:
            for row in column.split():
                if value in row:
                    return key
    return 'undefined'

In [195]:
keywords['region'] = keywords['keyword'].apply(region)

In [196]:
keywords.region.unique()

array(['undefined', 'Центр', 'Северо-Запад', 'Дальний Восток'],
      dtype=object)

In [200]:
keywords.query('region == "Центр"')

,keyword,shows,region
127,авито москва,979292,Центр
849,авито ярославль,209581,Центр
1259,москва,153050,Центр
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
...,...,...,...
98943,вакансии на авито москва от прямых работодателей,3738,Центр
99292,афиша кино москва на сегодня,3704,Центр
99590,авито ярославль автомобили с пробегом,3673,Центр
99890,яндекс метро москва,4548,Центр


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1 В переменную years запишите список из всех годов с 1950 по 2010.

2 Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов   содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3 Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4 Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [278]:
ratings = pd.read_csv(path+"ratings.csv")

In [279]:
movies = pd.read_csv(path+"movies.csv")

In [280]:
years = []
for i in range (1950, 2011):
    years.append(str(i))

In [281]:
def year_movies(column):
    for i in re.findall('\d{4}', column):
        if i in years:
            return i
    return '1900'

In [282]:
movies['year'] = movies['title'].apply(year_movies)

In [283]:
ratings =ratings.groupby(['movieId'])['rating'].mean().reset_index()

In [284]:
total = movies.merge(ratings[['movieId','rating']], on = 'movieId',how = 'outer')

In [287]:
total = total.fillna(0)

In [290]:
total.groupby(['year'])['rating'].mean().sort_values(ascending =False)

year
1951    3.906653
1962    3.772962
1973    3.688212
1953    3.635020
1957    3.603241
          ...   
1956    3.174243
1900    3.164852
1990    3.162096
2003    3.140829
2001    3.135890
Name: rating, Length: 62, dtype: float64